<a href="https://colab.research.google.com/github/mdeevan/LightweightFineTuning/blob/main/LightweightFineTuning_bert_base_ucase.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lightweight Fine-Tuning Project

TODO: In this cell, describe your choices for each of the following

* PEFT technique:
* Model:
* Evaluation approach:
* Fine-tuning dataset:

## Loading and Evaluating a Foundation Model

TODO: In the cells below, load your chosen pre-trained Hugging Face model and evaluate its performance prior to fine-tuning. This step includes loading an appropriate tokenizer and dataset.

### Install libraries

In [1]:
!pip install accelerate --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 21.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 29.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 33.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━

In [2]:
!pip install transformers --upgrade
!pip install evaluate seqeval
!pip install peft
# !pip install bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 21.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.8 MB/s eta 0:00:00
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16161 sha256=68b4330d1d223284608a0ed31072d339f79f67ed0afa932a8c01f4b4bbac870a
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.9/190.9 kB 5.9 MB/s eta 0:00:00


#### Import Libraries

In [3]:
from transformers import (AutoModelForSequenceClassification,
                          AutoTokenizer, DataCollatorWithPadding,
                          TrainingArguments, Trainer,
#                           BitsAndBytesConfig
                         )

from datasets     import load_dataset

from peft import (LoraConfig, get_peft_model, TaskType,
#                   LoftQConfig, prepare_model_for_kbit_training
                 )


import torch
import evaluate
import numpy as np
import pandas as pd


In [4]:
# https://www.evidentlyai.com/classification-metrics/multi-class-metrics

accuracy  = evaluate.load('accuracy')
f1        = evaluate.load('f1')
precision = evaluate.load('precision')
recall    = evaluate.load('recall')

In [5]:
# import numpy as np
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)

    return accuracy.compute(predictions=predictions, references=labels)

In [6]:
checkpoint = "google-bert/bert-base-uncased"
data_file = "financial_phrasebank"
data_file_subset = "sentences_66agree"

### Load Dataset

In [7]:
from datasets import load_dataset

raw_dataset = load_dataset(data_file,
                           name=data_file_subset,
                           split="train").train_test_split(test_size=0.2,
                                                           shuffle=True,
                                                           seed=42)

raw_train = raw_dataset.pop('train')
raw_train_valid = raw_train.train_test_split(test_size=.1, shuffle=True, seed=42)
raw_dataset['train'] = raw_train_valid.pop('train')
raw_dataset['eval'] = raw_train_valid.pop('test')
raw_dataset


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/4217 [00:00<?, ? examples/s]

DatasetDict({
    test: Dataset({
        features: ['sentence', 'label'],
        num_rows: 844
    })
    train: Dataset({
        features: ['sentence', 'label'],
        num_rows: 3035
    })
    eval: Dataset({
        features: ['sentence', 'label'],
        num_rows: 338
    })
})

In [8]:
labels = raw_dataset["train"].features['label'].names
labels

['negative', 'neutral', 'positive']

In [9]:
label2id = {l:i for i, l in enumerate(labels)}
id2label = {i:l for i, l in enumerate(labels)}

In [10]:
input_max_length = max([len(s) for s in raw_dataset['train']['sentence']])
input_max_length

315

In [11]:
print(label2id)
print(id2label)
print(len(label2id))

{'negative': 0, 'neutral': 1, 'positive': 2}
{0: 'negative', 1: 'neutral', 2: 'positive'}
3


## Load Model, Tokenizer, Collator

#### Load Model

In [12]:
# # Quantize the model

# quant_config=BitsAndBytesConfig(load_in_4bit = True,
#                                 bnb_4bit_quant_type="nf4",
#                                 bnb_4bit_use_double_quant=True,
#                                 bnb_4bit_compute_dtype=torch.bfloat16
#                                 )

model = AutoModelForSequenceClassification.from_pretrained(checkpoint,
                                                           num_labels = len(label2id),
                                                           id2label=id2label,
                                                           label2id=label2id,
#                                                            quantization_config=quant_config,
                                                          )

# model = prepare_model_for_kbit_training(model)

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

print("device = ", device)

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


device =  cuda


#### Load Tokenizer

In [13]:
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [14]:
# print("model     = ", model)
print("tokenizer = ", tokenizer)

tokenizer =  BertTokenizerFast(name_or_path='google-bert/bert-base-uncased', vocab_size=30522, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}


In [15]:
def tokenize_function(data):
    return tokenizer(data['sentence'],
#                      max_length=input_max_length,
                     truncation=True,
#                      padding='max_length'
                    )


tokenized_dataset = raw_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/844 [00:00<?, ? examples/s]

Map:   0%|          | 0/3035 [00:00<?, ? examples/s]

Map:   0%|          | 0/338 [00:00<?, ? examples/s]

In [16]:
tokenized_dataset

DatasetDict({
    test: Dataset({
        features: ['sentence', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 844
    })
    train: Dataset({
        features: ['sentence', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 3035
    })
    eval: Dataset({
        features: ['sentence', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 338
    })
})

#### Collator

In [17]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer,
                                        padding=True,
#                                         padding='max_length',
#                                         max_length=input_max_length)
                                       )

In [18]:
data_collator

DataCollatorWithPadding(tokenizer=BertTokenizerFast(name_or_path='google-bert/bert-base-uncased', vocab_size=30522, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}, padding=True, max_length=None, pad_to_multiple_of=None, return_tensors='pt')

In [19]:
[len(x) for x in tokenized_dataset['train'][:10]['sentence']]

[120, 75, 152, 46, 152, 90, 50, 167, 160, 49]

In [20]:
[len(x) for x in tokenized_dataset['train'][:10]['input_ids']]

[22, 19, 38, 12, 35, 20, 14, 57, 42, 16]

#### Evaluate model before fine-tuning

In [21]:

def evaluate_samples(model=model, ds=tokenized_dataset['train'], sample_start=0, sample_count=10):
  samples = ds[sample_start : sample_start+sample_count]

  samples = {k: v for k, v in samples.items() if k not in ['sentence', 'label']}

  batch = data_collator(samples ).to(device)

  output = model(**batch).logits

  predictions=torch.argmax(output, dim=1).cpu().numpy()

  return predictions


In [22]:
sample_start = 20
sample_count = 10

predictions=evaluate_samples(model, tokenized_dataset['train'], sample_start, sample_count)
references =tokenized_dataset['train']['label'][sample_start:sample_start+sample_count]
print("predictions = {}".format(predictions.tolist()))
print("references  = {}".format(references))

predictions = [0, 0, 0, 0, 0, 0, 0, 1, 0, 0]
references  = [1, 2, 2, 1, 2, 0, 1, 1, 1, 2]


In [23]:


clf =  evaluate.combine(["accuracy",'f1','precision','recall'])
accuracy_metric = accuracy.compute (predictions = predictions, references  = references )
f1_metric       = f1.compute       (predictions = predictions, references  = references,  average = "macro")
precision_metric= precision.compute(predictions = predictions, references  = references,   average = "macro", zero_division=0)
recall_metric   = recall.compute   (predictions = predictions, references  = references,  average = "macro")


print(accuracy_metric)
print(f1_metric)
print(precision_metric)
print(recall_metric)

{'accuracy': 0.2}
{'f1': 0.17777777777777778}
{'precision': 0.3703703703703704}
{'recall': 0.39999999999999997}


In [24]:
model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

## Performing Parameter-Efficient Fine-Tuning

TODO: In the cells below, create a PEFT model from your loaded model, run a training loop, and save the PEFT model weights.

In [25]:
# config = LoraConfig()

# loftq_config = LoftQConfig(loftq_bits=4)

lora_config = LoraConfig(task_type=TaskType.SEQ_CLS ,
                         inference_mode    = False,
#                          init_lora_weights = "loftq",
#                          loftq_config      = loftq_config,
                         r                 = 64,
                         lora_alpha        = 128,
                         lora_dropout      = 0.05,
                         bias              = 'none',
                         target_modules    = ['query','value', 'key',"all-linear"], # 'out_proj'],
                         modules_to_save   = ['classifier']
                         )


In [26]:
lora_model = get_peft_model(model, lora_config)

In [27]:
lora_model.print_trainable_parameters()

trainable params: 3,541,251 || all params: 113,025,798 || trainable%: 3.1331351449515976


#### Training Loop

In [28]:
Training_Arguments = TrainingArguments(
    per_device_train_batch_size = 4,
    per_device_eval_batch_size  = 4,
    output_dir                  = "bert_peft_classifier",
    learning_rate               = 2e-5,
    num_train_epochs            = 20,
    weight_decay                = 0.005,
    save_strategy               = 'epoch',
    evaluation_strategy         = 'epoch',
    deepspeed                   = False,
    load_best_model_at_end      = True)

In [30]:
trainer = Trainer(
                  model=lora_model,
                  args=Training_Arguments,
                  train_dataset=tokenized_dataset['train'],
                  eval_dataset =tokenized_dataset['eval'],
                  compute_metrics=compute_metrics,
                  tokenizer=tokenizer,
                  data_collator=data_collator,
)

In [31]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.876400,0.606458,0.757396
2,0.467900,0.375184,0.881657
3,0.402000,0.356744,0.893491
4,0.355700,0.340100,0.908284
5,0.300800,0.339883,0.917160
6,0.292000,0.339799,0.920118
7,0.285500,0.334346,0.923077
8,0.248000,0.339513,0.931953
9,0.212600,0.348634,0.928994
10,0.223700,0.407077,0.923077


TrainOutput(global_step=15180, training_loss=0.258912346454163, metrics={'train_runtime': 861.5593, 'train_samples_per_second': 70.454, 'train_steps_per_second': 17.619, 'total_flos': 1485003689259300.0, 'train_loss': 0.258912346454163, 'epoch': 20.0})

In [33]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [34]:
saved_checkpoint = '/content/drive/MyDrive/ftMLC-Bert-Mar-19-00'

In [35]:
trainer.save_model(saved_checkpoint)

## Performing Inference with a PEFT Model

TODO: In the cells below, load the saved PEFT model weights and evaluate the performance of the trained PEFT model. Be sure to compare the results to the results from prior to fine-tuning.

In [36]:
trainer2 = Trainer(
                  model=lora_model,
                  args=Training_Arguments,
                  train_dataset=tokenized_dataset['train'],
                  eval_dataset =tokenized_dataset['test'],
                  compute_metrics=compute_metrics,
                  tokenizer=tokenizer,
                  data_collator=data_collator
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [37]:
print(trainer2.evaluate())

{'eval_loss': 0.42112115025520325, 'eval_accuracy': 0.9028436018957346, 'eval_runtime': 8.9276, 'eval_samples_per_second': 94.539, 'eval_steps_per_second': 23.635}


In [48]:

sample_start=0
sample_count=4

inferences = []
references = []
for i in range(sample_start, 844,  sample_count):
  inferences += evaluate_samples(lora_model, tokenized_dataset['test'], sample_start, sample_count).tolist()
  references += tokenized_dataset['test']['label'][sample_start:sample_start+sample_count]

  # print (i)

# inferences = evaluate_samples(peft_model, tokenized_dataset['test'], sample_start, sample_count).tolist()
# print(inferences)

# references=tokenized_dataset['test']['label'][sample_start:sample_start+sample_count]
# print(references)

In [51]:
mismatches = []
pred_vs_ref = []
for n, (i, r) in enumerate(zip(inferences, references)):
  txt = "prediction: {}, reference: {}, sentence:{}".format(id2label[i], id2label[r], raw_dataset['test'][n]['sentence'])
  pred_vs_ref.append(txt)

  if i!=r:
    # mismatches.append(n)
    txt = "prediction: {}, reference: {}, sentence:{}".format(id2label[i], id2label[r], raw_dataset['test'][n]['sentence'])
    mismatches.append(txt)

In [52]:
print(len(mismatches)/len(inferences))
print(len(pred_vs_ref))

0.0
844


In [53]:
mismatches

[]

In [54]:
pred_vs_ref

['prediction: neutral, reference: neutral, sentence:Simultaneously , Alma Media has purchased a 35 % share of Arena Interactive , a subsidiary of Arena Partners with a focus on mobile solutions development .',
 'prediction: positive, reference: positive, sentence:The third order awarded to Outokumpu Technology is by Shalkiya Zinc of Kazakhstan for the Shalkiya zinc-lead project in Kazakhstan .',
 "prediction: neutral, reference: neutral, sentence:The new location is n't the only change Wellmont has in store for its air transport service .",
 'prediction: negative, reference: negative, sentence:Operating profit for the 12-month period decreased from EUR157 .5 m , while net sales increased from EUR634 .3 m , as compared to 2007 .',
 'prediction: neutral, reference: neutral, sentence:The value of the confirmed aircraft orders totals EUR 2bn .',
 "prediction: positive, reference: positive, sentence:The company 's model is based on developer contracting construction projects and customer fo